# 读取保存的历代.npy文件，包含全部X与F，预备进行相关性分析
**独立代码块，需要自定义chkpt_dir与n_gen**

In [ ]:
%matplotlib widget

import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
matplotlib.rcParams["font.sans-serif"] = ["SimHei"]  
matplotlib.rcParams["axes.unicode_minus"] = False    # 正常显示负号
from pymoo.indicators.hv import HV

chkpt_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\C23参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\NSGA完整迭代计算结果\0812-NSGA2-Seed2-100群200代"

n_gen = 200  # 总迭代轮次
n_obj = 3   # 目标维度
n_var = 12  # 设计变量(参数)维度

all_X_history = []  # 用于收集所有代次的 X
all_F_history = []  # 用于收集所有代次的 F

for gen in range(1, n_gen + 1):
    # 注意这里 filename 要用 f"generation_{gen}.npz"
    filename = os.path.join(chkpt_dir, f"generation_{gen}.npz")
    
    # 读取 .npz 文件
    with np.load(filename) as data:
        X = data["X"]  # shape = (pop_size, n_var)
        F = data["F"]  # shape = (pop_size, n_obj)
    
    # 将当前代的数据放入 list
    all_X_history.append(X)
    all_F_history.append(F)

# 现在 all_X_history 和 all_F_history 都是 list，每个元素是 (pop_size, n_var) or (pop_size, n_obj)
# 如果想把所有代次堆成一个大的 2D 矩阵，可以用 np.vstack:

all_X_history = np.vstack(all_X_history)  # shape = (n_gen * pop_size, n_var)
all_F_history = np.vstack(all_F_history)  # shape = (n_gen * pop_size, n_obj)

print("all_X_history shape:", all_X_history.shape)
print("all_F_history shape:", all_F_history.shape)

# 可视化 {X1,…,X12}与 {Y1,Y2,Y3}之间的相关系数子矩阵
**依赖于X与F的.npy数据导入**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

param_names = [f"X{i+1}" for i in range(12)]
target_names = [f"Y{i+1}" for i in range(3)]

df_X = pd.DataFrame(all_X_history, columns=param_names)
df_F = pd.DataFrame(all_F_history, columns=target_names)

# 计算一个完整的相关矩阵(15x15)，然后只取想看的 12x3 或 3x12 子矩阵
df_all = pd.concat([df_X, df_F], axis=1)

corr_matrix = df_all.corr(method="pearson")  # 形状(15, 15)

# 只截取 X1~X12 与 Y1~Y3 的部分, 这会是形状 (12, 3)
sub_corr = corr_matrix.loc[param_names, target_names]

print("子矩阵 shape:", sub_corr.shape)
print(sub_corr)

# 可视化, 让 x 轴显示 Y1~Y3, y 轴显示 X1~X12
plt.figure(figsize=(6, 8))
sns.heatmap(sub_corr, annot=True, cmap="coolwarm", center=0)
plt.title("Correlation of X1..X12 vs. Y1..Y3")
plt.show()


# 计算 Sobol 主效应 (S1) 和全效应 (ST)，并绘制指数热力图
**依赖于X与F的.npy数据导入**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# SALib 1.5.1+ 官方建议使用 `sobol.sample` 而不是 `saltelli.sample`
from SALib.sample import sobol
from SALib.analyze import sobol as sobol_analyze

# ============ 1. 载入已有数据 ============
X = all_X_history  
Y = all_F_history  
print("X shape:", X.shape)  # (5808, 12)
print("Y shape:", Y.shape)  # (5808, 3)

# ============ 2. 多输出随机森林训练 ============
# 拆分训练集、测试集 (80% 训练 + 20% 测试)
# test_size = 0.2 表示划分方式，可自行调大或调小
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.2,
                                                    random_state=42)
# 构造多输出随机森林
# 重要参数解释:
# - n_estimators=100: 随机森林中树的数量，越多越稳定但计算更慢
# - max_depth=None: 默认为None表示树可以不受限地生长，视数据量而定
# - random_state=42: 随机种子，保证结果可复现
rf = RandomForestRegressor(n_estimators=100, 
                           max_depth=None, 
                           random_state=42)
print("[Info] 开始训练多输出随机森林...")
rf.fit(X_train, Y_train)  # Y_train.shape=(n_samples, 3) 多输出回归
# 预测测试集
Y_pred = rf.predict(X_test)  # shape=(test_size, 3)
# 分别计算每个目标的 MSE
mse_list = []
for j in range(Y.shape[1]):
    mse_j = mean_squared_error(Y_test[:, j], Y_pred[:, j])
    mse_list.append(mse_j)
    print(f"RandomForest for Y{j+1}, Test MSE = {mse_j:.4f}")

# ============ 3. 全局灵敏度分析 (Sobol) ============
# 3.1 定义输入参数范围 (bounds)
# 这里简单取历史数据的最小值 & 最大值，也可以使用工程先验信息
X_min = X.min(axis=0)  # shape=(12,)
X_max = X.max(axis=0)  # shape=(12,)
problem = {
    'num_vars': 12,
    'names': param_names,
    'bounds': [[X_min[i], X_max[i]] for i in range(12)]
}

# 采样数量(越大结果越稳定, 但计算量增加)
# 注意 Sobol 序列收敛性最好使用 2^n, 例如 512, 1024, 2048
N_base = 256  
print("[Info] 使用 Sobol 采样, N_base =", N_base)
param_values = sobol.sample(problem, N_base, calc_second_order=True)
print("param_values shape:", param_values.shape)

# 3.2 评估代理模型 (rf) 在采样点的预测值
# 由于是多输出, Y_sobol_pred.shape = (param_values.shape[0], 3)
Y_sobol_pred = rf.predict(param_values)  

S1_list = []  # 用于存放3个目标的 S1 (each is shape=(12,))
ST_list = []  # 用于存放3个目标的 ST

# 3.3 分别对每个目标做 Sobol 指数分析
for j in range(Y.shape[1]):
    print(f"\n=== Sobol Analysis for Y{j+1} ===")
    # 当前目标在所有采样点的值
    current_output = Y_sobol_pred[:, j]

    # 做 Sobol 分析
    Si = sobol_analyze.analyze(problem, current_output, calc_second_order=True)

    # 打印主效应 (S1) 和全效应 (ST)
    # 注意: S1, ST 都是长度12的数组, 对应 X1..X12
    s1 = Si['S1']  # shape=(12,)
    st = Si['ST']  # shape=(12,)
    print("Sobol First Order Indices (S1):", s1)
    print("Sobol Total Effect Indices (ST):", st)

    # 保存到列表
    S1_list.append(s1)
    ST_list.append(st)

# 将list转换为矩阵, shape=(3, 12) => 行=目标, 列=参数
S1_array = np.array(S1_list)  # shape=(3,12)
ST_array = np.array(ST_list)  # shape=(3,12)

# 转置后, shape=(12,3) => 行=参数 X1..X12, 列=目标 Y1..Y3
S1_array_t = S1_array.T
ST_array_t = ST_array.T

# 画热力图（行 = X, 列 = Y）
# 以全效应 (ST)为例：
param_names = [f"X{i+1}" for i in range(12)]  # 行标签
target_names = [f"Y{i+1}" for i in range(3)]  # 列标签

plt.figure(figsize=(8, 6))

sns.heatmap(ST_array_t,       # shape=(12,3)
            annot=True,       # 在格子中显示数值
            cmap="coolwarm",  # 颜色方案，可换成 "bwr", "RdBu_r", etc.
            vmin=0, vmax=0.8,  
            xticklabels=target_names,
            yticklabels=param_names
           )

plt.title("Sobol ST Indices (X in rows, Y in columns)")
plt.xlabel("Objectives")
plt.ylabel("Parameters")
plt.show()


# 分析不同型式双牵引拉杆对于动力学性能的影响

## Step 1: 读取数据 & 构造 D、可视化字体设置

In [ ]:
# 读取数据并构造 D（<0 为 Λ 型；≥0 为 V/一般型）
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# ==== 显式可改：Excel 文件路径与工作表名 ====
FILE  = Path(r"F:/ResearchMainStream/0.ResearchBySection/C.动力学模型/C23参数优化/参数优化实现/ParallelSweepSimpack/结果分析组/后处理-NSGA23算法效能对比/merged_nsga_23nd.xlsx")
SHEET = "merged_nsga_23nd"   # 若名称不同，改这里

df = pd.read_excel(FILE, sheet_name=SHEET)

# ==== 列名重命名（可根据文件调整映射）====
# 以如下表头为例: X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 F0 F1 F2
rename_map = {
    "X2":  "Kpx",
    "X10": "Lx1",
    "X11": "Lx2",
    "X12": "Lx3",
    "X8":  "Cld",
    "X9":  "Chx",
    "F1":  "Wear",  # 磨耗数 (wear number)
    "F0":  "Y1",    # 临界速度 critical speed，定义为 Y1 = -Vc[m/s]
    "F2":  "Y3",    # 平稳性指标 (ride comfort index)
}

df = df.rename(columns=rename_map)

# 只保留本次分析需要列，去缺失
use_cols = ["Kpx", "Lx1", "Lx2", "Lx3", "Cld", "Chx", "Wear", "Y1", "Y3"]
df = df[use_cols].dropna().copy()

# 几何差值 D；并派生 Vc(km/h) 用于阅读（Y1 是 -Vc[m/s]）
df["D"] = df["Lx1"] - df["Lx2"]
df["Vc_kmh"] = (-df["Y1"]) * 3.6

# 中文字体设置
plt.rcParams["font.sans-serif"] = ["SimHei", "Microsoft YaHei", "Arial Unicode MS"]
plt.rcParams["axes.unicode_minus"] = False

print(df.describe(include="all").T)


## Step 2: 偏相关——在相同控制变量下，计算 r(D, Y | controls) 与 p 值
D 与磨耗（Y2）的散点 + 皮尔逊/斯皮尔曼相关

In [ ]:
from scipy.stats import pearsonr, spearmanr

r_p = pearsonr(df["D"], df["Wear"]).statistic
r_s = spearmanr(df["D"], df["Wear"]).statistic

plt.figure()
plt.scatter(df["D"], df["Wear"], s=20, alpha=0.8)
plt.axvline(0, linestyle="--", linewidth=1)  # D=0 分界线：左侧为 Λ 型
plt.xlabel("D = Lx1 - Lx2（<0 为 Λ 型）")
plt.ylabel("磨耗数 (Y2)")
plt.title(f"D 与磨耗 (Y2) | Pearson r={r_p:.3f}  Spearman ρ={r_s:.3f}")
plt.grid(True, linewidth=0.3, alpha=0.6)
# plt.savefig("scatter_D_vs_Wear.png", dpi=220, bbox_inches="tight")
plt.show()


## Step 3: OLS(HC3) 显著性检验：Y ~ D + Kpx + Cld + Chx + Lx3

In [ ]:
import statsmodels.api as sm

# 回归：Y2（磨耗）
X = df[["D", "Kpx", "Cld", "Chx", "Lx3"]].astype(float)
X = sm.add_constant(X)
y = df["Wear"].astype(float)

res_y2 = sm.OLS(y, X).fit(cov_type="HC3")  # 异方差稳健标准误
print(res_y2.summary())

# 计算 D 的 partial R^2（由 t 值换算）；同时给出标准化系数
t_D = res_y2.tvalues["D"]
df_resid = res_y2.df_resid
partial_R2_y2 = (t_D*t_D) / (t_D*t_D + df_resid)

# 标准化系数（z-score 后再回归）
Z = df[["Wear","D","Kpx","Cld","Chx","Lx3"]].apply(lambda s: (s - s.mean())/s.std(ddof=0))
res_y2_std = sm.OLS(Z["Wear"], sm.add_constant(Z[["D","Kpx","Cld","Chx","Lx3"]])).fit()
beta_std_y2 = res_y2_std.params["D"]

print(f"\n[Y2] coef(D)={res_y2.params['D']:.4g}, t={t_D:.3f}, p={res_y2.pvalues['D']:.3g}, "
      f"partial R^2={partial_R2_y2:.3f}, std_beta={beta_std_y2:.3f}")

# 同样地做 Y1（-Vc，数值越小=速度越高）
y1 = df["Y1"].astype(float)
res_y1 = sm.OLS(y1, X).fit(cov_type="HC3")
t_D1 = res_y1.tvalues["D"]; pr2_y1 = (t_D1*t_D1)/(t_D1*t_D1 + res_y1.df_resid)
Z1 = df[["Y1","D","Kpx","Cld","Chx","Lx3"]].apply(lambda s: (s - s.mean())/s.std(ddof=0))
res_y1_std = sm.OLS(Z1["Y1"], sm.add_constant(Z1[["D","Kpx","Cld","Chx","Lx3"]])).fit()
print(f"\n[Y1=-Vc] coef(D)={res_y1.params['D']:.4g}, t={t_D1:.3f}, p={res_y1.pvalues['D']:.3g}, "
      f"partial R^2={pr2_y1:.3f}, std_beta={res_y1_std.params['D']:.3f}")

# Y3（平稳性指标）
y3 = df["Y3"].astype(float)
res_y3 = sm.OLS(y3, X).fit(cov_type="HC3")
t_D3 = res_y3.tvalues["D"]; pr2_y3 = (t_D3*t_D3)/(t_D3*t_D3 + res_y3.df_resid)
Z3 = df[["Y3","D","Kpx","Cld","Chx","Lx3"]].apply(lambda s: (s - s.mean())/s.std(ddof=0))
res_y3_std = sm.OLS(Z3["Y3"], sm.add_constant(Z3[["D","Kpx","Cld","Chx","Lx3"]])).fit()
print(f"\n[Y3] coef(D)={res_y3.params['D']:.4g}, t={t_D3:.3f}, p={res_y3.pvalues['D']:.3g}, "
      f"partial R^2={pr2_y3:.3f}, std_beta={res_y3_std.params['D']:.3f}")


## Step 4: 偏相关：r(D, Y | Kpx, Cld, Chx, Lx3)

In [ ]:
# 通过“残差化”做偏相关：先用控制变量回归掉 D 和 Y 的线性影响，再对残差做相关
from scipy.stats import pearsonr

def partial_corr_by_residual(df, ycol):
    # 对 y 与 D 分别用 controls 做 OLS，取残差
    controls = sm.add_constant(df[["Kpx","Cld","Chx","Lx3"]].astype(float))
    ry = sm.OLS(df[ycol].astype(float).values, controls.values).fit().resid
    rD = sm.OLS(df["D"].astype(float).values,       controls.values).fit().resid
    r, p = pearsonr(rD, ry)
    return r, p, len(df) - controls.shape[1] - 1

for ycol, yname in [("Wear","Y2 磨耗"), ("Y1","Y1 = -Vc"), ("Y3","Y3 平稳性")]:
    r, p, dof = partial_corr_by_residual(df, ycol)
    print(f"[偏相关] r(D, {yname} | Kpx,Cld,Chx,Lx3) = {r:.3f}, p={p:.3g}, dof={dof}")

## Step 5: Λ 型 vs V/一般型 的 Welch t 检验 + 箱线图

In [ ]:
from scipy.stats import ttest_ind

mask_lambda = df["D"] < 0
mask_vgen   = ~mask_lambda

def group_t(ycol, label):
    a = df.loc[mask_lambda, ycol].astype(float)
    b = df.loc[mask_vgen,   ycol].astype(float)
    t, p = ttest_ind(a, b, equal_var=False)
    print(f"[Welch] {label}: n_Λ={len(a)}, mean_Λ={a.mean():.2f} | n_V={len(b)}, mean_V={b.mean():.2f} | t={t:.2f}, p={p:.3g}")

for col, name in [("Wear","Y2 磨耗"), ("Y1","Y1=-Vc（数值越小=速度越高）"), ("Y3","Y3 平稳性")]:
    group_t(col, name)

# 箱线图（对比三目标）
plt.figure()
plt.boxplot([df.loc[mask_lambda,"Wear"], df.loc[mask_vgen,"Wear"]], labels=["Λ 型(D<0)","V/一般型(D≥0)"])
plt.ylabel("磨耗 (Y2)")
plt.title("Λ 型 vs V/一般型 | 磨耗分布对比")
plt.grid(True, linewidth=0.3, alpha=0.6)
# plt.savefig("box_wear_groups.png", dpi=220, bbox_inches="tight")
plt.show()


## Step 6: RandomForest 回归偏依赖图（1D + 2D）

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay

X_pd = df[["Kpx", "D"]].values
y_pd = df["Wear"].values

rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
rf.fit(X_pd, y_pd)

# 1D：Y2 对 D 的偏依赖
disp1 = PartialDependenceDisplay.from_estimator(
    rf, X_pd, [1],
    feature_names=["Kpx","D"],
    grid_resolution=40,
    percentiles=(0.01, 0.99)
)
disp1.axes_[0, 0].set_title("偏依赖：Y2(磨耗) 对 D")
disp1.figure_.tight_layout()
# disp1.figure_.savefig("pd_Y2_vs_D.png", dpi=220, bbox_inches="tight")

# 2D PDP：Y2 对 (Kpx, D)
disp2 = PartialDependenceDisplay.from_estimator(
    rf, X_pd, [(0, 1)],
    feature_names=["Kpx", "D"],
    grid_resolution=32,
    percentiles=(0.02, 0.98),
)
disp2.axes_[0, 0].set_title("偏依赖：Y2(磨耗) 对 (Kpx, D)")
disp2.figure_.tight_layout()
# disp2.figure_.savefig("pd_Y2_vs_Kpx_D.png", dpi=220, bbox_inches="tight")

plt.show()

## Step 7: 基于 RF 的等高线图：更直观看“低 Kpx + Λ 型(D<0) 是低磨耗带”

In [ ]:
# 直接用 RF 在网格上预测生成等高线
kpx_min, kpx_max = np.quantile(df["Kpx"], [0.01, 0.99])
d_min,   d_max   = np.quantile(df["D"],   [0.01, 0.99])

gx = np.linspace(kpx_min, kpx_max, 120)
gy = np.linspace(d_min,   d_max,   120)
GX, GY = np.meshgrid(gx, gy)
YY = rf.predict(np.c_[GX.ravel(), GY.ravel()]).reshape(GX.shape)

plt.figure()
cs = plt.contourf(GX, GY, YY, levels=14)
plt.colorbar(cs, label="预测磨耗 Y2")
plt.xlabel("Kpx")
plt.ylabel("D = Lx1 - Lx2（<0 为 Λ 型）")
plt.title("随机森林代理下的 Y2 等高线（直观观察低磨耗带）")
plt.grid(True, linewidth=0.3, alpha=0.6)
# plt.savefig("contour_Y2_Kpx_D.png", dpi=220, bbox_inches="tight")
plt.show()


# Markdown 占位

In [ ]:
# 代码占位